Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [5]:
# windowing function
def create_windows(data, window_size, horizon, target_col_idx=0, shuffle=False):
    X, y = [], []
    if data.ndim == 1:
        data = data[:, np.newaxis]

    for i in range(len(data) - window_size - horizon + 1):
        x_i = data[i:i + window_size]
        y_i = data[i + window_size + horizon - 1, target_col_idx]
        X.append(x_i)
        y.append(y_i)

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [6]:
past, future = (5, 2)
X_train, y_train = create_windows(train_data, past, future)
X_test, y_test = create_windows(test_data, past, future)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [7]:
normalizer = keras.layers.Normalization()
normalizer.adapt(X_train)

inputs = keras.layers.Input(shape=(past, 1))
norm_inputs = normalizer(inputs)
gru_1 = keras.layers.GRU(64, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2)(norm_inputs)
gru_2 = keras.layers.GRU(64, dropout=0.2, recurrent_dropout=0.2)(gru_1)
outputs = keras.layers.Dense(1)(gru_2)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization (Normalization)   │ (None, 5, 1)           │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 64)          │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,892 (148.02 KB)

 Trainable params: 37,889 (148.00 KB)

 Non-trainable params: 3 (16.00 B)

In [8]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - loss: 119.3631 - mae: 10.2077 - val_loss: 15.2857 - val_mae: 3.1078
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 15.7960 - mae: 3.1748 - val_loss: 13.6245 - val_mae: 2.9266
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 13.6479 - mae: 2.8929 - val_loss: 11.8658 - val_mae: 2.7017
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 12.6457 - mae: 2.7732 - val_loss: 10.4794 - val_mae: 2.5301
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 11.2721 - mae: 2.6051 - val_loss: 9.7096 - val_mae: 2.4323
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 10.5657 - mae: 2.5332 - val_loss: 9.4175 - val_mae: 2.3863
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 10.7453 - mae: 2.5514 - val_loss: 8.9287 - val_mae: 2.3084
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 10.0093 - mae: 2.4918 - val_loss: 8.8220 - val_mae: 2.2935
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s

In [9]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.9318 - mae: 2.0619
Test Loss: [6.9029388427734375, 2.0545971393585205]


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [10]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0

df['year_sin'] = np.sin(2 * np.pi * df['portion_year'])
df['year_cos'] = np.cos(2 * np.pi * df['portion_year'])

df['month'] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df['dayofweek'] = df['Date'].dt.dayofweek
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

df_multi = df[['Temp', 'portion_year', 'year_sin', 'year_cos',
               'month_sin', 'month_cos', 'dow_sin', 'dow_cos']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [11]:
## Create windows
X_train, y_train = create_windows(train_data.to_numpy(),
                                  past, future, target_col_idx=0)
X_test, y_test = create_windows(test_data.to_numpy(),
                                past, future, target_col_idx=0)

In [12]:
normalizer = keras.layers.Normalization()
normalizer.adapt(X_train)

inputs = keras.layers.Input(shape=(past, 8))
norm_inputs = normalizer(inputs)
gru_1 = keras.layers.GRU(128, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2)(norm_inputs)
gru_2 = keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2)(gru_1)
outputs = keras.layers.Dense(1)(gru_2)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_1 (Normalization) │ (None, 5, 8)           │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 5, 128)         │        52,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,210 (594.57 KB)

 Trainable params: 152,193 (594.50 KB)

 Non-trainable params: 17 (72.00 B)

In [13]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - loss: 107.2134 - mae: 9.4748 - val_loss: 10.4425 - val_mae: 2.5658
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 10.0043 - mae: 2.5106 - val_loss: 7.9446 - val_mae: 2.1846
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 9.0915 - mae: 2.4049 - val_loss: 7.8277 - val_mae: 2.1757
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 8.2012 - mae: 2.2773 - val_loss: 8.0109 - val_mae: 2.1966
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 8.7314 - mae: 2.3351 - val_loss: 7.5077 - val_mae: 2.1154
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 7.7465 - mae: 2.2314 - val_loss: 7.7467 - val_mae: 2.1616
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 8.0406 - mae: 2.2711 - val_loss: 7.7223 - val_mae: 2.1322
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 7.6630 - mae: 2.1914 - val_loss: 8.0432 - val_mae: 2.1981
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step 

In [14]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.1384 - mae: 1.9565
Test Loss: [6.228333473205566, 1.950204610824585]


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


---
La arquitectura 'many-to-one' es apta para A(Clasificación de sentimiento en textos), B(Verificación de voz para iniciar el ordenador) y C(Un clasificador que clasifique piezas de música según su autor). Esto es así puesto que son varios los respectivos inputs y un solo output (cadena de texto--> sentimiento, cadena de audio-->orden, cadena de audio-->autor).

Por el contrario, C(Generación de música) no es apta para esta arquitectura puesto se espera como salida una cadena de audio.

---

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


---
Todas las opciones anteriores son correctas. En efecto, al usar one-hot encoding se generan vectores tan largos como la longitud del diccionario, siendo todos los vectores igual de diferentes entre sí. Al usar word embeddings se obtienen vectores de una longitud fija cuya cercanía representa similitud entre palabras (a, b y c). Además, como a estos vectores se le asigna una dimensión fija se podría aplicar una técnica de reducción de componentes como PCA para realizar agrupaciones semánticas entre las palabras (d).

---